<a href="https://colab.research.google.com/github/seungyeon38/Webtoon_recommendation_system/blob/master/TF-IDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [80]:
import numpy as np
import pandas as pd
import string
string.punctuation
from math import log
import nltk
nltk.download('stopwords')
import re
from tqdm import tqdm


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [71]:
comic = pd.read_csv('./sample_data/Webtoon Dataset.csv')

In [72]:
#defining the function to remove punctuation
def remove_punctuation(text):
    punctuationfree_list = []

    for word in text:
      if word in string.punctuation:
        punctuationfree_list.append(" ")
      else :
        punctuationfree_list.append(word)
    return "".join(punctuationfree_list)

# def remove_punctuation(text):
#     punctuationfree = "".join([i for i in text if i not in string.punctuation])
#     return punctuationfree

  # punctuationfree = "".join([i for i in text if i not in string.punctuation])


comic['Clean_Summary'] = comic['Summary'].apply(lambda x:remove_punctuation(x))

#setting lower case
comic['Lower_Summary'] = comic['Clean_Summary'].apply(lambda x: x.lower())

In [73]:
# #defining function for tokenization
# def tokenization(text):
#     tokens = re.split('W+', text)
#     return tokens

#defining function for tokenization
p = re.compile('[a-z]+')

def tokenization(text):
    # print(text)
    result = p.findall(text)
    # print(result)
    return result

#applying function to the column

comic['Tokenied_Summary'] = comic['Lower_Summary'].apply(lambda x: tokenization(x))


In [74]:
#stop words present in the library
stopwords = nltk.corpus.stopwords.words('english')

#defining the function to remove stopwords from tokenized text
def remove_stopwords(text):
    for word in text:
      output= [word for word in text if word not in stopwords]
    return output
    
#applying the function
comic['No_Stopwords_Summary'] = comic['Tokenied_Summary'].apply(lambda x:remove_stopwords(x))


In [75]:
df1 = comic['No_Stopwords_Summary']
doc_words_list = df1.values.tolist()

docs = []

for doc_words in doc_words_list :
  docs.append(" ".join(doc_words))


# docs.shape
# type(docs) #pandas.core.frame.DataFrame
# df1.head()
# docs = [
#   '먹고 싶은 사과',
#   '먹고 싶은 바나나',
#   '길고 노란 바나나 바나나',
#   '저는 과일이 좋아요'
# ] 
# vocab = list(set(w for doc in docs for w in doc.split()))
vocab_list = list(set(w for doc in doc_words_list for w in doc))
vocab_list.sort()


In [76]:
docs_num = len(docs) # 총 문서의 수

def tf(t, d):
    return d.count(t)

def idf(t):
    df = 0
    for doc in docs:
        df += t in doc
    return log(docs_num/(df + 1))

def tfidf(t, d):
    return tf(t,d)* idf(t)

In [77]:
result = []
total_result = [0 for i in range(len(vocab_list))]

for i in range(docs_num): # 각 문서에 대해서 아래 명령을 수행
    result.append([])
    doc = docs[i]
    for j in range(len(vocab_list)):
        vocab = vocab_list[j]        
        result[-1].append(tf(vocab, doc)) # 각 문서에서 각 단어의 수를 세어서 result list에 넣음 
        total_result[j] += tf(vocab, doc)

print(total_result)

total_result_1_num = 0
total_result_big_num = 0

for i in total_result:
  if i==1:
    total_result_1_num += 1 
  elif i>20:
    total_result_big_num += 1
    
print('total_result_1_num')
print(total_result_1_num) # 2530 -> 2997

print('total_result_big_num')
print(total_result_big_num) # 481 -> 113

tf_ = pd.DataFrame(result, columns = vocab_list)
tf_
# tf_.shape #(569, 5696) -> (569, 5443)


[176, 5, 1, 1, 3, 10, 22, 78, 1, 1, 1, 2, 1, 3, 1, 2, 454, 7, 10, 3, 3, 1, 1, 3, 1, 1, 23, 10, 1, 1, 1, 1, 107, 6, 1, 1, 13, 105, 3, 14, 3, 1, 2, 6, 3, 2, 13, 10, 1, 1, 4, 1, 4, 1, 1, 1, 1, 2, 1, 1, 1, 3, 1, 1, 4, 1, 1, 7, 2, 1, 2, 1, 1, 30, 2, 14, 1, 1, 2, 1, 64, 6, 1, 2, 2, 5, 4, 1, 1, 1, 1, 1, 6, 1, 1, 1, 1, 1, 106, 7, 1, 5, 23, 2, 26, 1, 5, 3, 250, 1, 2, 2, 4, 36, 2, 6, 2, 1, 1, 1, 2, 4, 100, 3, 10, 5, 4, 12, 1, 1, 1, 1, 4, 3, 101, 4, 8, 24, 7, 4, 18, 5, 1, 1, 1, 4, 1, 1, 22, 2, 1, 1, 1, 2, 3, 1, 5, 1, 1, 1, 6, 1, 5, 2, 1, 1, 1, 3, 25, 2, 1, 1, 16, 1, 9, 6, 4, 6, 1, 33, 8, 5, 1, 12, 7, 2, 1, 2, 3, 1, 1, 18, 5, 2, 6, 12, 1, 1, 32, 4, 6, 1, 1, 6, 18, 1, 3, 13, 8, 1, 11, 5, 85, 2, 1, 17, 7, 3, 5, 1, 2, 1, 1, 2, 4, 3, 1, 1, 1, 1, 1, 1, 20, 1, 1, 2, 1, 1, 18, 49, 3, 1, 1, 1, 5, 4, 1, 1, 37, 26, 2, 3, 1, 1, 2, 205, 3, 1, 8, 28, 10, 1, 24, 3, 1, 2, 2, 34, 2, 8, 2, 6, 2, 1, 95, 1, 6, 1, 2, 1, 1, 1, 4, 2, 6, 1, 1, 1, 1, 1, 1, 1, 45, 1, 1, 1, 13, 2, 1, 5, 11, 2, 4, 5, 1, 2, 3, 1, 1, 6, 1, 1,

,ab,abandoned,abbey,abbott,abel,abilities,ability,able,abode,abroad,absolutely,absorb,absorbs,absurd,absurdly,abuse,ac,academy,accept,acceptance,accepted,accepting,acception,access,accessibility,accessing,accident,accidentally,accidents,accompanied,accomplish,account,ace,achieve,acquired,acres,across,act,acting,action,...,yeti,yibak,yigang,yiheon,yijae,yijun,yohan,yomanville,yong,yoo,york,young,younger,youngest,youth,yu,yuhyeon,yuli,yumi,yun,yunha,yunjo,yuri,yuseong,yuuki,yuzuki,zahn,zamsa,zealand,zecalion,zedekiah,zephyr,zero,zeronis,zodiac,zoe,zombie,zombies,zombified,zone
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
565,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
566,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
567,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [78]:
result = []
for j in range(len(vocab_list)):
    vocab = vocab_list[j]
    result.append(idf(vocab))

idf_ = pd.DataFrame(result, index = vocab_list, columns = ["IDF"])
idf_

,IDF
ab,1.360274
abandoned,4.552121
abbey,5.650733
abbott,5.650733
abel,5.245268
...,...
zoe,5.245268
zombie,4.041295
zombies,5.245268
zombified,5.650733


In [81]:
result = []
for i in tqdm(range(docs_num)):
    result.append([])
    d = docs[i]
    for j in range(len(vocab_list)):
        vocab = vocab_list[j]
        result[-1].append(tfidf(vocab,d))

tfidf_ = pd.DataFrame(result, columns = vocab_list)
tfidf_

100%|██████████| 569/569 [08:19<00:00,  1.14it/s]


,ab,abandoned,abbey,abbott,abel,abilities,ability,able,abode,abroad,absolutely,absorb,absorbs,absurd,absurdly,abuse,ac,academy,accept,acceptance,accepted,accepting,acception,access,accessibility,accessing,accident,accidentally,accidents,accompanied,accomplish,account,ace,achieve,acquired,acres,across,act,acting,action,...,yeti,yibak,yigang,yiheon,yijae,yijun,yohan,yomanville,yong,yoo,york,young,younger,youngest,youth,yu,yuhyeon,yuli,yumi,yun,yunha,yunjo,yuri,yuseong,yuuki,yuzuki,zahn,zamsa,zealand,zecalion,zedekiah,zephyr,zero,zeronis,zodiac,zoe,zombie,zombies,zombified,zone
0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.727109,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,4.39797,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.139188,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.139188,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.727109,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.790004,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,1.360274,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.727109,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
565,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.727109,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
566,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
567,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

In [103]:
print(result[500])
# print(len(result))

result_dictionary_list = []

# for i in range(len(result)):
#    result_dictionary = []
#    result_dictionary.append({k: v for v, k in enumerate(result[i])})
#    result_dictionary_list.append(result_dictionary)

# print(result_dictionary_list[0])

# print(dict(zip(result[0],range(len(result[0])))))


# for i in result[0]:
# print(sorted(result[0], reverse = True))


[1.3602738124179945, 0.0, 0.0, 5.6507332535663855, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.6507332535663855, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.6507332535663855, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,